# XGBoost : La technique de modélisation la plus précise pour les données structurées

Dans ce tutoriel, vous apprendrez à construire et optimiser des modèles avec le **gradient boosting**. Cette méthode domine de nombreuses compétitions Kaggle et offre des résultats de pointe sur une variété de jeux de données.

---


# Introduction
Dans une grande partie de ce cours, vous avez fait des prédictions avec la méthode des forêts aléatoires, qui offrent de meilleures performances qu'un seul arbre de décision simplement en moyennant les prédictions de plusieurs arbres.

Nous appelons la méthode des forêts aléatoires une méthode d'ensemble (ensemble method). Par définition, les méthodes d'ensemble combinent les prédictions de plusieurs modèles (par exemple, plusieurs arbres, dans le cas des forêts aléatoires).

Ensuite, nous allons apprendre une autre méthode d'ensemble appelée **gradient boosting**.

---

# Gradient Boosting
Le gradient boosting est une méthode qui fonctionne en ajoutant des modèles dans un ensemble de manière itérative.

### 1. Initialisation : 
L'ensemble commence avec un seul modèle, dont les prédictions peuvent être très basiques. (Même si ses prédictions sont très inexactes, les ajouts suivants corrigeront ces erreurs.)

### 2. Cycle itératif :
- Utilisez l'ensemble actuel pour générer des prédictions pour chaque observation dans les données.
- Calculez une fonction de perte (comme l'erreur quadratique moyenne) avec ces prédictions.
- Ajustez un nouveau modèle pour réduire la perte en déterminant ses paramètres via la descente de gradient.
- Ajoutez ce nouveau modèle à l'ensemble et recommencez le cycle.

---

### Exemple
Nous commençons par charger les données d'entraînement et de validation dans X_train, X_valid, y_train, et y_valid.

Dans cet exemple, vous utiliserez la bibliothèque XGBoost, qui signifie **"extreme gradient boosting"**. C'est une implémentation de gradient boosting avec des fonctionnalités supplémentaires axées sur la performance et la rapidité.

**Code de base pour XGBoost :**

In [12]:
pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/124.9 MB ? eta -:--:--
    --------------------------------------- 2.4/124.9 MB 16.8 MB/s eta 0:00:08
   -- ------------------------------------- 6.6/124.9 MB 19.2 MB/s eta 0:00:07
   --- ------------------------------------ 12.3/124.9 MB 22.7 MB/s eta 0:00:05
   ---- ----------------------------------- 14.9/124.9 MB 19.6 MB/s eta 0:00:06
   ------ --------------------------------- 20.7/124.9 MB 21.5 MB/s eta 0:00:05
   -------- ------------------------------- 27.3/124.9 MB 23.7 MB/s eta 0:00:05
   ----------- ---------------------------- 35.4/124.9 MB 25.8 MB/s eta 0:00:04
   -------------- ------------------------- 44.0/124.9 MB 28.0 MB/s eta 0:00:03
   ---------------- ----------------------- 50.9/124.9 MB 28.7 MB/s eta 0:00:03
   ------------------ --------------------- 58.5/124.9 MB 29.6 MB/s eta 0:00:03
   -------------------- ------------------- 65.0/124.

In [8]:
# Importation des bibliothèques nécessaires
from sklearn.compose import ColumnTransformer  # Pour transformer les colonnes spécifiques
from sklearn.impute import SimpleImputer  # Pour gérer les valeurs manquantes
from sklearn.preprocessing import OneHotEncoder  # Pour encoder les colonnes catégoriques
from sklearn.pipeline import Pipeline  # Pour créer un pipeline d'étapes
from sklearn.model_selection import train_test_split  # Pour diviser les données en ensembles d'entraînement et de validation
from sklearn.metrics import mean_absolute_error  # Pour évaluer le modèle avec l'erreur absolue moyenne
from xgboost import XGBRegressor  # Modèle de régression XGBoost
import pandas as pd  # Pour manipuler les données

# Étape 1 : Chargement des données
# Le chemin du fichier CSV contenant les données de Melbourne
melbourne_file_path = 'melb_data.csv'
melbourne_data = pd.read_csv(melbourne_file_path)

# Étape 2 : Préparation des données
# Supprimer les lignes où la cible ('Price') est manquante
melbourne_data.dropna(axis=0, subset=['Price'], inplace=True)

# Séparer la cible (y) des prédicteurs (X)
y = melbourne_data['Price']  # Variable cible
X = melbourne_data.drop(['Price'], axis=1)  # Données prédicteurs

# Identifier les colonnes numériques et catégoriques dans X
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Étape 3 : Transformation des données
# Création d'un transformateur pour gérer les colonnes numériques et catégoriques
preprocessor = ColumnTransformer(
    transformers=[
        # Pour les colonnes numériques : remplacer les valeurs manquantes par la moyenne
        ('num', SimpleImputer(strategy='mean'), numerical_cols),
        # Pour les colonnes catégoriques : appliquer un encodage OneHot en ignorant les catégories inconnues
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
    ]
)

# Étape 4 : Création du pipeline
# Le modèle utilisé dans ce pipeline est XGBRegressor
model = XGBRegressor(random_state=0)  # Régression XGBoost avec un paramètre d'aléatoire fixé
my_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Étape de prétraitement
    ('model', model)  # Étape de modélisation
])

# Étape 5 : Division des données
# Diviser les données en ensembles d'entraînement et de validation
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size=0.8, test_size=0.2, random_state=0)

# Étape 6 : Entraînement du pipeline
# Le pipeline applique le prétraitement et entraîne le modèle en une seule commande
my_pipeline.fit(X_train, y_train)

# Étape 7 : Prédiction et évaluation
# Prédire les valeurs cibles pour l'ensemble de validation
predictions = my_pipeline.predict(X_valid)

# Calculer l'erreur absolue moyenne (Mean Absolute Error)
print("Erreur absolue moyenne : " + str(mean_absolute_error(predictions, y_valid)))


Erreur absolue moyenne : 164091.43196566642


---

**Code de base pour XGBoost :**

```python
from xgboost import XGBRegressor

# Modèle initial
my_model = XGBRegressor()
my_model.fit(X_train, y_train)

# Évaluation
from sklearn.metrics import mean_absolute_error

predictions = my_model.predict(X_valid)
print("Erreur absolue moyenne : " + str(mean_absolute_error(predictions, y_valid)))
```


---

### Paramètres à ajuster dans XGBoost

#### **n_estimators**
- Spécifie combien de fois le cycle de modélisation est exécuté (nombre de modèles ajoutés à l'ensemble).
- Valeurs typiques : 100-1000 (selon la taille des données et la valeur de `learning_rate`).

Exemple :
```python
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train)
```

---

#### **early_stopping_rounds**
- Interrompt automatiquement l'entraînement si le score de validation ne s'améliore plus après un certain nombre d'itérations.
- Utilise un ensemble de validation pour calculer les scores (`eval_set`).

Exemple avec arrêt anticipé :
```python
my_model = XGBRegressor(n_estimators=500)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
```

---

#### **learning_rate**
- Contrôle l'impact de chaque modèle dans l'ensemble.
- Une petite valeur de `learning_rate` combinée à un grand `n_estimators` donne des modèles plus précis mais prend plus de temps à entraîner.
- Valeur par défaut : `learning_rate=0.1`.

Exemple :
```python
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
```

---

#### **n_jobs**
- Permet d'utiliser le parallélisme pour accélérer l'entraînement.
- Configurez `n_jobs` au nombre de cœurs de votre machine.

Exemple :
```python
my_model = XGBRegressor(n_estimators=1000, learning_rate=0.05, n_jobs=4)
my_model.fit(X_train, y_train, 
             early_stopping_rounds=5, 
             eval_set=[(X_valid, y_valid)], 
             verbose=False)
```

---

### Conclusion

XGBoost est une bibliothèque puissante pour les données tabulaires (celles stockées dans des DataFrames Pandas). Avec un ajustement minutieux des paramètres, vous pouvez entraîner des modèles très précis.

---

### À vous de jouer !

Entraînez votre propre modèle avec XGBoost dans le prochain exercice ! 😊